In [80]:
import pandas as pd
import numpy as np
import string
import re
import pickle

from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.initializers import Constant
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

In [14]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "cp1252")
df

,Emotion,ID,Date,Query,Author,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [15]:
# removing hashtags and mentions
def remove_hashatags_and_mentions(text):
    return re.sub(r'#\w*', r' ', re.sub(r'@\w*', r' ', str(text)))

df.Text = df.Text.apply(remove_hashatags_and_mentions)
df

,Emotion,ID,Date,Query,Author,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"http://twitpic.com/2y1zl - Awww, that's a bu..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,I dived many times for the ball. Managed to ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"no, it's not behaving at all. i'm mad. why a..."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [16]:
# removing punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df.Text = df.Text.apply(remove_punctuation)
df

,Emotion,ID,Date,Query,Author,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,httptwitpiccom2y1zl Awww thats a bummer Yo...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his Facebook by t...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,I dived many times for the ball Managed to s...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,no its not behaving at all im mad why am i h...
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up Having no school is the best feel...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDBcom Very cool to hear old Walt intervie...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover Ask me fo...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time Tup...


In [17]:
# prepare data for training

untokenized_texts = np.array(df.Text)
emotions = np.array(df.Emotion)

In [5]:
english_embedding_dict = {}
with open("glove.twitter.27B.200d.txt", "r", encoding="utf-8") as english_glove:
    for line in tqdm(english_glove):
        values = line.split()
        word = remove_punctuation(values[0])
        vectors = np.asarray(values[1:], 'float32')
        english_embedding_dict[word] = vectors

1193514it [01:13, 16135.53it/s]


In [18]:
EMBEDDING_LEN = 200

In [30]:
word_cnt = dict()
for text in tqdm(untokenized_texts):
    for word in text.split():
        word_cnt[word] = word_cnt.get(word, 0) + 1
clean_texts = []
for text in tqdm(untokenized_texts):
    clean_text = ' '.join([word for word in text.split() if word_cnt[word] > 10])
    clean_texts.append(clean_text)

100%|████████████████████████████████████████████████████████████████████| 1600000/1600000 [00:05<00:00, 306333.52it/s]


In [34]:
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(clean_texts)
padded_sequences = pad_sequences(tokenizer.texts_to_sequences(clean_texts), padding = 'post')
max_len = padded_sequences.shape[1]
word_index = tokenizer.word_index

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_LEN))
for word, i in tqdm(word_index.items()):
    embedding_matrix[i] = english_embedding_dict.get(word, np.zeros(EMBEDDING_LEN))

100%|████████████████████████████████████████████████████████████████████████| 29342/29342 [00:00<00:00, 376184.91it/s]


In [50]:
model = Sequential()
model.add(Embedding(
    len(word_index) + 1,
    EMBEDDING_LEN,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=max_len,
    trainable=False,
))
model.add(LSTM(500))
model.add(Dropout(0.1))
model.add(Dense(500))
model.add(Dropout(0.1))
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(learning_rate=1e-4), metrics=["accuracy"])

In [51]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 200)           5868600   
_________________________________________________________________
lstm_3 (LSTM)                (None, 500)               1402000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 500)               250500    
_________________________________________________________________
dropout_7 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 501       
Total params: 7,521,601
Trainable params: 1,653,001
Non-trainable params: 5,868,600
____________________________________

In [53]:
texts_train, texts_test, values_train, values_test = train_test_split(padded_sequences, emotions, test_size = 0.85)

In [57]:
model.fit(texts_train, values_train, batch_size=512, epochs=1000) # , validation_data=(texts_test, values_test))

Epoch 1/1000
469/469 [==============================] - 43s 93ms/step - loss: 1.3294 - accuracy: 0.3533
Epoch 2/1000
469/469 [==============================] - 43s 92ms/step - loss: 1.3013 - accuracy: 0.3590: 36s - loss: 1.2 - ETA: 30s -  - ETA: 27s  -  - ETA: 20s - loss: 1.291 - ETA: 18 - ETA: 15s - loss: 1.2933 - accuracy: 0
Epoch 3/1000
469/469 [==============================] - 43s 92ms/step - loss: 1.2818 - accuracy: 0.3639: 31s - loss: 1 - ETA: 0s - loss: 1.280
Epoch 4/1000
469/469 [==============================] - 43s 92ms/step - loss: 1.2482 - accuracy: 0.37024s - - ETA: 1s - loss:
Epoch 5/1000
469/469 [==============================] - 43s 92ms/step - loss: 1.2320 - accuracy: 0.3723
Epoch 6/1000
469/469 [==============================] - 44s 93ms/step - loss: 1.2001 - accuracy: 0.3776
Epoch 7/1000
469/469 [==============================] - 45s 96ms/step - loss: 1.1921 - accuracy: 0.3794
Epoch 8/1000
469/469 [==============================] - 45s 96ms/step - loss: 1.1555 - acc

469/469 [==============================] - 45s 97ms/step - loss: 0.5862 - accuracy: 0.4662
Epoch 74/1000
469/469 [==============================] - 46s 97ms/step - loss: 0.5848 - accuracy: 0.4659
Epoch 75/1000
469/469 [==============================] - 46s 97ms/step - loss: 0.5696 - accuracy: 0.4670
Epoch 76/1000
469/469 [==============================] - 45s 97ms/step - loss: 0.5698 - accuracy: 0.4666
Epoch 77/1000
469/469 [==============================] - 46s 99ms/step - loss: 0.5728 - accuracy: 0.4661
Epoch 78/1000
469/469 [==============================] - 45s 96ms/step - loss: 0.5783 - accuracy: 0.4660
Epoch 79/1000
469/469 [==============================] - 46s 98ms/step - loss: 0.5698 - accuracy: 0.4664
Epoch 80/1000
469/469 [==============================] - 45s 95ms/step - loss: 0.5649 - accuracy: 0.4671
Epoch 81/1000
469/469 [==============================] - 46s 98ms/step - loss: 0.5509 - accuracy: 0.4685
Epoch 82/1000
469/469 [==============================] - 46s 97ms/ste

469/469 [==============================] - 43s 93ms/step - loss: 0.4299 - accuracy: 0.4746
Epoch 134/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.4192 - accuracy: 0.4759: 41s - loss: 0.4109 - accuracy: 0. - ETA: 41s - loss: 0.404 - ETA:  - ETA: 4s - loss: - ETA: 1s - los
Epoch 135/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.4146 - accuracy: 0.4755: 39s - loss: 0.4008 - ac - ETA: 37s - loss: 0.3893 - accurac - ETA: 37s - loss: - ETA: 30s - loss: - ETA: 28s - loss: 0.4049 - accuracy: 0. - ETA: - ETA: 15s - loss: 0.4117 - accuracy: 0. - ETA: - ETA: 0s - loss: 0.4153 
Epoch 136/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.4137 - accuracy: 0.4757: 42s - loss: 0 - ETA: 39s - loss: 0.3760 - - E - ETA: 5s - loss: 0.4128  - ETA: 4s - loss: 0.4128 
Epoch 137/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.4090 - accuracy: 0.4761
Epoch 138/1000
469/469 [==============================] - 43s 9

469/469 [==============================] - 45s 95ms/step - loss: 0.2772 - accuracy: 0.4825
Epoch 201/1000
469/469 [==============================] - 45s 95ms/step - loss: 0.2853 - accuracy: 0.48190s - loss: 0.2849 - accura
Epoch 202/1000
469/469 [==============================] - 45s 96ms/step - loss: 0.2702 - accuracy: 0.4828
Epoch 203/1000
469/469 [==============================] - 46s 98ms/step - loss: 0.2779 - accuracy: 0.4823
Epoch 204/1000
469/469 [==============================] - 45s 95ms/step - loss: 0.2740 - accuracy: 0.4830
Epoch 205/1000
469/469 [==============================] - 45s 97ms/step - loss: 0.2768 - accuracy: 0.4822
Epoch 206/1000
469/469 [==============================] - 45s 96ms/step - loss: 0.2762 - accuracy: 0.4824
Epoch 207/1000
469/469 [==============================] - 45s 95ms/step - loss: 0.2659 - accuracy: 0.4833
Epoch 208/1000
469/469 [==============================] - 45s 96ms/step - loss: 0.2648 - accuracy: 0.4832
Epoch 209/1000
469/469 [===========

469/469 [==============================] - 43s 92ms/step - loss: 0.2171 - accuracy: 0.4856: 37s - loss: - ETA: 31s  - ETA: 28s  - ETA: 25s -  - ETA: 22s - loss: 0.2097 - accurac - ETA: 21s - loss: 0.2093 - accuracy:  - ET - ETA: 17s - l
Epoch 265/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.2148 - accuracy: 0.4858: 26s - lo - ETA: 24s - loss: 0.2183 - accuracy: 0. - ETA: 23s - loss: 0.2184 - accur
Epoch 266/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.2124 - accuracy: 0.4861
Epoch 267/1000
469/469 [==============================] - 43s 93ms/step - loss: 0.2126 - accuracy: 0.4860
Epoch 268/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.2120 - accuracy: 0.4858
Epoch 269/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.2087 - accuracy: 0.4862: 11s - loss: 0.2031 - accuracy: 0. - ETA: 10s - loss: 0.2032 - accuracy: 0
Epoch 270/1000
469/469 [==============================] - 43s 92ms/step -

...

469/469 [==============================] - 43s 93ms/step - loss: 0.1161 - accuracy: 0.4918: 21s - loss: 0.1165 - ac - ETA: 19s - - ETA: 16s - loss: 0.
Epoch 603/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1154 - accuracy: 0.4915
Epoch 604/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1115 - accuracy: 0.4918
Epoch 605/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1102 - accuracy: 0.4920
Epoch 606/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1117 - accuracy: 0.4918
Epoch 607/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1136 - accuracy: 0.4917: 37s - loss: 0.1036 - accuracy: - ETA: 37s - loss: 0.105 - ETA: 35s - loss: 0.1051 - accuracy: 0.490 - ETA: 35s - loss: 0.1044 - accurac - ETA: 34
Epoch 608/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1173 - accuracy: 0.4913: 11s - loss: 0.1161 - accura - ET - ETA: 2s - loss: 0.1166 - accur

469/469 [==============================] - 46s 97ms/step - loss: 0.1085 - accuracy: 0.49221s - loss: 0.1084 -  - ETA: 1s - loss: 0
Epoch 664/1000
469/469 [==============================] - 45s 95ms/step - loss: 0.1088 - accuracy: 0.4920
Epoch 665/1000
469/469 [==============================] - 45s 95ms/step - loss: 0.1098 - accuracy: 0.4919
Epoch 666/1000
469/469 [==============================] - 45s 95ms/step - loss: 0.1101 - accuracy: 0.4919
Epoch 667/1000
469/469 [==============================] - 46s 97ms/step - loss: 0.1104 - accuracy: 0.4919
Epoch 668/1000
469/469 [==============================] - 46s 98ms/step - loss: 0.1063 - accuracy: 0.4923
Epoch 669/1000
469/469 [==============================] - 45s 96ms/step - loss: 0.1064 - accuracy: 0.4923
Epoch 670/1000
469/469 [==============================] - 46s 97ms/step - loss: 0.1094 - accuracy: 0.4920
Epoch 671/1000
469/469 [==============================] - 45s 96ms/step - loss: 0.1128 - accuracy: 0.4917
Epoch 672/1000
469/46

469/469 [==============================] - 44s 93ms/step - loss: 0.1004 - accuracy: 0.49260s - loss: 0.1000 - accuracy
Epoch 738/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1083 - accuracy: 0.49190s - loss: 0.1085 - accuracy: 0.49 - ETA: 0s - loss: 0.1085 - accu
Epoch 739/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1058 - accuracy: 0.4923
Epoch 740/1000
469/469 [==============================] - 43s 93ms/step - loss: 0.1035 - accuracy: 0.4925
Epoch 741/1000
469/469 [==============================] - 43s 93ms/step - loss: 0.1104 - accuracy: 0.4916
Epoch 742/1000
469/469 [==============================] - 45s 95ms/step - loss: 0.1040 - accuracy: 0.4923
Epoch 743/1000
469/469 [==============================] - 43s 92ms/step - loss: 0.1050 - accuracy: 0.4922
Epoch 744/1000
469/469 [==============================] - 44s 94ms/step - loss: 0.1036 - accuracy: 0.4923: 17s - - ETA: 
Epoch 745/1000
469/469 [==============================] - 

469/469 [==============================] - 45s 96ms/step - loss: 0.0944 - accuracy: 0.4925: 42s - loss: 0.0772 - accuracy: 0. - ETA: 38 - ETA: 34s - 
Epoch 801/1000
469/469 [==============================] - 44s 94ms/step - loss: 0.0923 - accuracy: 0.49276s - loss: 0.0903  - ETA: 
Epoch 802/1000
469/469 [==============================] - 45s 96ms/step - loss: 0.0959 - accuracy: 0.4925
Epoch 803/1000
469/469 [==============================] - 44s 95ms/step - loss: 0.0960 - accuracy: 0.4924: 16s - loss: 0.0920 - accu
Epoch 804/1000
469/469 [==============================] - 45s 96ms/step - loss: 0.0980 - accuracy: 0.4922: 40s - loss: 0. - ETA: 38s - loss: 0.0930 - accurac - ETA: 3 - ETA - ETA: 25s - loss: 0.0978 - a - ETA: 24s - loss: 0.0992 - accur - ETA: 23s - loss: 0.0982 - accuracy: 0. - ETA: 23s -  - ETA: 11s -  - ETA: 9s - - ETA: 3s -
Epoch 805/1000
469/469 [==============================] - 44s 93ms/step - loss: 0.0936 - accuracy: 0.49252s - loss: 0.0937 - accura
Epoch 806/1000
46

KeyboardInterrupt: 

In [64]:
def predict_emotion(text):
    model_input = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=max_len, padding="post")
    model_output = model.predict(model_input)[0][0]
    return f"Sentence is positive for {(model_output / 4) * 100} %"

In [79]:
predict_emotion("This party was an explosion")

'Sentence is positive for 99.01155233383179 %'

In [82]:
with open('tokenizer_1_6B_set.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

model.save('model_1_6B_set_LSTM_1_hidden_layer')

INFO:tensorflow:Assets written to: model_1_6B_set_LSTM_1_hidden_layer\assets


INFO:tensorflow:Assets written to: model_1_6B_set_LSTM_1_hidden_layer\assets
